In [ ]:
# import os, pathlib, requests, msal

# CLIENT_ID = "ecbaaa8e-adcb-48b2-be6a-903686f124e5"
# AUTHORITY = "https://login.microsoftonline.com/consumers"  # Hotmail/MSA
# SCOPES = ["Files.Read"]  # don't include openid/profile/offline_access
# GRAPH = "https://graph.microsoft.com/v1.0"
# OUTPUT_DIR = "downloaded_share"
# TARGET_NAME_SUBSTR = "Lameness Videos"  # match on name/webUrl if needed
# TARGET_OWNER_HINT = "negar_sadr_ubc_ca"  # optional: part of owner/upn/webUrl

# def msal_token():
#     app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)
#     result = app.acquire_token_silent(SCOPES, account=None)
#     if not result:
#         flow = app.initiate_device_flow(scopes=SCOPES)
#         if "user_code" not in flow:
#             raise RuntimeError("Failed to create device flow")
#         print(flow["message"])  # sign in as sarah_s96@hotmail.com
#         result = app.acquire_token_by_device_flow(flow)
#     if "access_token" not in result:
#         raise RuntimeError(f"Auth failed: {result}")
#     return result["access_token"]

# def session_with_token():
#     s = requests.Session()
#     s.headers.update({"Authorization": f"Bearer {msal_token()}"})
#     return s

# def pick_shared_root(sess):
#     # List items shared with your Hotmail
#     r = sess.get(f"{GRAPH}/me/drive/sharedWithMe")
#     if r.status_code != 200:
#         print("sharedWithMe failed:", r.status_code, r.text); r.raise_for_status()
#     hits = r.json().get("value", [])
#     # Print candidates for debugging
#     for i, it in enumerate(hits):
#         name = it.get("name"); webUrl = it.get("webUrl", "")
#         owner = (it.get("remoteItem", {}).get("createdBy", {})
#                   .get("user", {}).get("displayName", "")) or ""
#         print(f"[{i}] {name} | owner:{owner} | {webUrl}")

#     # Heuristics to find your folder
#     for it in hits:
#         name = (it.get("name") or "").lower()
#         webUrl = (it.get("webUrl") or "").lower()
#         if TARGET_NAME_SUBSTR.lower() in name or TARGET_NAME_SUBSTR.lower() in webUrl:
#             if TARGET_OWNER_HINT and TARGET_OWNER_HINT.lower() not in webUrl:
#                 continue
#             # Resolve to driveId/itemId
#             remote = it.get("remoteItem", {})
#             drive_id = remote.get("parentReference", {}).get("driveId")
#             item_id  = remote.get("id")
#             if drive_id and item_id:
#                 return drive_id, item_id, it.get("name")
#     raise RuntimeError("Could not find the shared folder in /me/drive/sharedWithMe output.")

# def list_children(sess, drive_id, item_id):
#     url = f"{GRAPH}/drives/{drive_id}/items/{item_id}/children"
#     while url:
#         r = sess.get(url)
#         if r.status_code != 200:
#             print("children failed:", r.status_code, r.text); r.raise_for_status()
#         data = r.json()
#         for c in data.get("value", []):
#             yield c
#         url = data.get("@odata.nextLink")

# def download_file(sess, item, dest_folder):
#     pathlib.Path(dest_folder).mkdir(parents=True, exist_ok=True)
#     name = item["name"]
#     dest = os.path.join(dest_folder, name)
#     # Skip if already fully downloaded (optional size check)
#     exp_size = item.get("size")
#     if exp_size and os.path.exists(dest) and os.path.getsize(dest) == exp_size:
#         print("✓ (skip) ", dest); return
#     content_url = f"{GRAPH}/drives/{item['parentReference']['driveId']}/items/{item['id']}/content"
#     with sess.get(content_url, stream=True) as s:
#         s.raise_for_status()
#         with open(dest, "wb") as f:
#             for chunk in s.iter_content(1024 * 1024):
#                 if chunk: f.write(chunk)
#     print("✓", dest)

# def walk(sess, drive_id, item, dest_folder):
#     if "folder" in item:
#         subdir = os.path.join(dest_folder, item["name"])
#         pathlib.Path(subdir).mkdir(parents=True, exist_ok=True)
#         for child in list_children(sess, drive_id, item["id"]):
#             walk(sess, drive_id, child, subdir)
#     else:
#         download_file(sess, item, dest_folder)

# def main():
#     sess = session_with_token()
#     drive_id, root_id, root_name = pick_shared_root(sess)
#     print("Resolved root:", root_name, "| drive:", drive_id, "| id:", root_id)
#     # Recurse from the root item
#     r = sess.get(f"{GRAPH}/drives/{drive_id}/items/{root_id}")
#     r.raise_for_status()
#     root_item = r.json()
#     out = os.path.join(OUTPUT_DIR, root_name or "shared_root")
#     pathlib.Path(out).mkdir(parents=True, exist_ok=True)
#     walk(sess, drive_id, root_item, out)
#     print("Done.")

# if __name__ == "__main__":
#     main()

In [ ]:
import cv2
import torch
import os
import numpy as np
import supervision as sv
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification
from PytorchWildlife import utils as pw_utils

SOURCE_VIDEO_PATH = '/Users/sarahabdelazim/Desktop/cow_catalyst/April 1/MVI_0114.MP4'
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)
TARGET_VIDEO_PATH = os.path.join(".","Users","sarahabdelazim","Desktop", "cow_catalyst", "April 1", "MVI_0114_processed.MP4")

# Initializing the MegaDetectorV6 model for image detection
# Valid versions are MDV6-yolov9-c, MDV6-yolov9-e, MDV6-yolov10-c, MDV6-yolov10-e or MDV6-rtdetr-c
detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="MDV6-yolov10-e")

# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
#detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")

# Initializing the AI4GOpossum model for image classification
classification_model = pw_classification.AI4GOpossum(device=DEVICE, pretrained=True)

In [ ]:
# import os, cv2, torch, supervision as sv
# from PytorchWildlife.models import detection as pw_detection

# SOURCE_VIDEO_PATH = "/Users/sarahabdelazim/Desktop/cow_catalyst/April 1/MVI_0114.MP4"
# TARGET_VIDEO_PATH = "/Users/sarahabdelazim/Desktop/cow_catalyst/April 1/MVI_0114_processed.mp4"

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# if DEVICE == "cuda":
#     torch.cuda.set_device(0)

# DET_CONF_THRESH = 0.40
# det_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="MDV6-yolov10-e")
# det_model.eval()

# cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
# if not cap.isOpened():
#     raise RuntimeError(f"Cannot open video: {SOURCE_VIDEO_PATH}")

# fps   = cap.get(cv2.CAP_PROP_FPS) or 25.0
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc= cv2.VideoWriter_fourcc(*"avc1")
# out   = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (width, height))

# box_annotator   = sv.BoxAnnotator()                      # draws rectangles only
# label_annotator = sv.LabelAnnotator(text_scale=0.6, text_thickness=1)  # draws text

# frame_idx = 0
# FRAME_SKIP = 0

# while True:
#     ok, frame_bgr = cap.read()
#     if not ok:
#         break

#     if FRAME_SKIP and (frame_idx % (FRAME_SKIP + 1) != 0):
#         out.write(frame_bgr); frame_idx += 1; continue

#     frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
#     res = det_model.single_image_detection(frame_rgb, det_conf_thres=DET_CONF_THRESH)

#     dets = res["detections"]      # sv.Detections
#     labels_txt = res["labels"]    # list[str], same order as dets

#     annotated = box_annotator.annotate(scene=frame_bgr.copy(), detections=dets)
#     annotated = label_annotator.annotate(scene=annotated, detections=dets, labels=labels_txt)

#     out.write(annotated)
#     frame_idx += 1

# cap.release()
# out.release()
# print(f"Saved annotated video → {TARGET_VIDEO_PATH}")

In [4]:
from pathlib import Path

phase3_dir = Path("/Users/sarahabdelazim/Library/CloudStorage/OneDrive-UBC")
print(phase3_dir.exists())
print(len(list(phase3_dir.rglob("*"))))  # how many items total

True
3542


In [ ]:
# importing libraries
import cv2

# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture(video_path)

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

# Read until video is completed
while(cap.isOpened()):
    
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
    # Display the resulting frame
        cv2.imshow('Frame', frame)
        
    # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

# Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()